# Proton Selection

In [ ]:
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import pickle
import os
from df_utils import merge_dfs
from utils import InFV
# plt.style.use('./presentation.mplstyle')

In [ ]:
savefig = False
figdir = "20240516"

# selection cuts
MAX_LEN_CUT = 25 
MIN_LEN_CUT = 25
MUSCORE_CUT = 40
PSCORE_CUT = 80

savepath = "/exp/sbnd/data/users/munjung/calibration/dfs/2024_prod_test"
dfname = "genie_cv.pkl"
calibdfname = "calib_df_muscore{}_pscore{}.pkl".format(MUSCORE_CUT, PSCORE_CUT)

trklabels = [r"stopping $p$", r"reinteracting $p$", r"$\nu$ induced $\mu$", r"cosmic $\mu$", "other"]
trkcolors = ["blue", "skyblue", "green", "lightgreen", "orange"]

In [ ]:
def var_breakdown(var, df):
    breakdown = [var[(df.mc_pdg == 2212) & (df.mc_end_process == 54)], var[(df.mc_pdg == 2212) & (df.mc_end_process != 54)], 
        var[(np.abs(df.mc_pdg) == 13) & (df.slc_mc_pdg != -1)], var[(np.abs(df.mc_pdg) == 13) & (df.slc_mc_pdg == -1)],
        var[(np.abs(df.mc_pdg) != 13) & (df.mc_pdg != 2212)]]
    return breakdown

In [ ]:
dfs = pickle.load(open(os.path.join(savepath, dfname), "rb"))
all_df = merge_dfs(dfs)

In [ ]:
# check that the CAF truth matching bug is fixed
var = all_df.slc_tmatch.eff
bins = np.linspace(0,1,21)
plt.hist(var, bins=bins)
plt.show();

In [ ]:
# https://github.com/SBNSoftware/sbnanaobj/blob/develop/sbnanaobj/StandardRecord/SREnums.h#L155-L221
var = all_df[(all_df.mc_pdg == 2212)].mc_end_process
for process in var.unique():
    print("proton", process, (var == process).sum()/len(var))

var = all_df[(np.abs(all_df.mc_pdg) == 211)].mc_end_process
for process in var.unique():
    print("pion", process, (var == process).sum()/len(var))

In [ ]:
slc_infv = all_df[InFV(all_df.slc_vertex)]
slc_dropbad = slc_infv[slc_infv.mc_pdg != -2147483648]

# select slices with 2 pfps
npfps = slc_dropbad.groupby(level=[0,1]).count().slc_self
twopfps_idx = npfps[npfps == 2].index 
twopfps = slc_dropbad.reset_index(level=2).loc[twopfps_idx]
twopfps = twopfps.reset_index().set_index(["entry", "rec.slc..index", "rec.slc.reco.pfp..index"])

# slice track multiplicities
twopfps["is_track"] = (twopfps.pfp.trackScore > 0.5)
twopfps["is_contained_track"] = (twopfps.pfp.trackScore > 0.5) & (twopfps.is_contained == True)
ntrk = twopfps.groupby(level=[0,1]).is_track.sum()
ntrkcont = twopfps.groupby(level=[0,1]).is_contained_track.sum()

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
var = [ntrk, ntrkcont]
bins = np.arange(0,4,1)
labels = ["all tracks", "contained tracks"]
plt.hist(var, bins=bins, label=labels, histtype="step")
plt.xlabel("Track Multiplicity")
plt.ylabel("Slices")
plt.legend()
plt.show();

In [ ]:
# select slices with 2 contained tracks
twoprong_slc_idx = ntrkcont[(ntrkcont == 2)].index
twoprong = slc_infv.reset_index().set_index(["entry", "rec.slc..index"]).loc[twoprong_slc_idx]
twoprong = twoprong.reset_index().set_index(["entry", "rec.slc..index", "rec.slc.reco.pfp..index"])

# longer / shorter prong
longer = twoprong.sort_values(by="len", ascending=False).groupby(level=[0,1]).nth(0)
shorter = twoprong.sort_values(by="len", ascending=False).groupby(level=[0,1]).nth(1)


In [ ]:
fig, ax = plt.subplots(figsize=(6,4))

trklen = twoprong.len
var = var_breakdown(trklen, twoprong)
bins = np.linspace(0,200,41)

plt.hist(var, bins=bins, 
         histtype="step", density=True,
         label=trklabels, 
         color=trkcolors)

plt.xlabel("Length [cm]")
plt.ylabel("Tracks")
plt.legend()

if savefig:
    plt.savefig(os.path.join("figures", figdir, "contained_track_len.pdf"), bbox_inches='tight')

In [ ]:
# cut on track lengths, could be different for each prong -- but keep same for now
maxlen = twoprong.groupby(level=[0,1]).max().len
minlen = twoprong.groupby(level=[0,1]).min().len
lencut_idx = maxlen[(maxlen > MAX_LEN_CUT) & (minlen > MIN_LEN_CUT)].index
lencut = twoprong.reset_index().set_index(["entry", "rec.slc..index"]).loc[lencut_idx]
lencut = lencut.reset_index().set_index(["entry", "rec.slc..index", "rec.slc.reco.pfp..index"])

# user the average across the 3 planes for chi2 scores 
lencut[("chi2_avg","chi2_muon")] = (lencut.chi2_I0.chi2_muon + lencut.chi2_I1.chi2_muon + lencut.chi2_I2.chi2_muon)/3
lencut[("chi2_avg","chi2_proton")] = (lencut.chi2_I0.chi2_proton + lencut.chi2_I1.chi2_proton + lencut.chi2_I2.chi2_proton)/3

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,4))
plt.subplots_adjust(wspace=0.25)

muscore = lencut.chi2_avg.chi2_muon
pscore = lencut.chi2_avg.chi2_proton
var_mu = var_breakdown(muscore, lencut)
var_p = var_breakdown(pscore, lencut)
bins_mu = np.linspace(0,80,41)
bins_p = np.linspace(0,300,41)

plt.subplot(1,2,1)
plt.hist(var_mu, bins=bins_mu, 
         stacked=True, 
         color=trkcolors)
plt.xlabel("$\\chi^2_{\\mu}$")
plt.ylabel("Tracks")

plt.subplot(1,2,2)
plt.hist(var_p, bins=bins_p, 
         stacked=True, 
         label=trklabels,
         color=trkcolors)
plt.xlabel("$\\chi^2_{p}$")
plt.ylabel("Tracks")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

if savefig:
    plt.savefig(os.path.join("figures", figdir, "chi2.pdf"), bbox_inches='tight')
plt.show();

In [ ]:
chi2cut = lencut[(muscore > MUSCORE_CUT) & (pscore < PSCORE_CUT)]
print("proton purity: ", (chi2cut.mc_pdg == 2212).sum()/len(chi2cut))
print("muon purity: ", (np.abs(chi2cut.mc_pdg) == 13).sum()/len(chi2cut))
print("pion purity: ", (np.abs(chi2cut.mc_pdg) == 211).sum()/len(chi2cut))

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))

trklen = chi2cut.len
var = var_breakdown(trklen, chi2cut)
for v in var:
    print("{} %".format(len(v)/len(trklen)*100))
bins = np.linspace(0,200,21)

n, bins, _ = plt.hist(var, bins=bins, 
                      stacked=True,
                      label=trklabels,
                      color=trkcolors)

plt.xlabel("Track Length [cm]")
plt.ylabel("Tracks")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

if savefig:
    plt.savefig(os.path.join("figures", figdir, "selected_trklen.pdf"), bbox_inches='tight')

plt.show();

In [ ]:
fig, ax = plt.subplots(figsize=(5,4))

tdir = chi2cut.dir.z
var = var_breakdown(tdir, chi2cut)
bins = np.linspace(-1,1,21)

plt.hist(var, bins=bins, 
         stacked=True, 
         label=trklabels,
         color=trkcolors)

plt.xlabel(r"cos$\theta_z$")
plt.ylabel("Tracks")
plt.title("$\\chi^2_{\\mu} > 20$ and $\\chi^2_{p} < 80$")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.text(0.05, 0.9, "proton purity: {:.2f}%".format(100*len(tdir[(chi2cut.mc_pdg == 2212)])/len(tdir)), transform=ax.transAxes)
for tdir_cut_val in [0.85, 0.9, 0.95]:
       tdir_cut = (tdir < tdir_cut_val)
       tdir_selected = chi2cut[tdir_cut]
       print("tdir cut: ", tdir_cut_val)
       plt.text(0.05, tdir_cut_val-0.15, r"cos$\theta_z$<{:.2f}".format(tdir_cut_val)+" proton purity: {:.2f}%".format((tdir_selected.mc_pdg == 2212).sum()/len(tdir_selected)*100), transform=ax.transAxes)

if savefig:
    plt.savefig(os.path.join("figures", figdir, "selected_dirz.pdf"), bbox_inches='tight')
plt.show();

# Save to nutple root file

In [ ]:
trkhits = dfs["hit"]
trks = dfs["pfp"]

selected_idx = chi2cut.index
print(len(selected_idx), "tracks selected")
trkhits = trkhits.reset_index().set_index(["entry", "rec.slc..index", "rec.slc.reco.pfp..index"]).loc[selected_idx]
trks = trks.loc[selected_idx]

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
varx = trkhits.rr
vary = trkhits.dqdx
bins = (np.linspace(0,100,41), np.linspace(0,5e3,21))
plt.hist2d(varx, vary, bins=bins)
plt.colorbar()
plt.show();

In [ ]:
# drop first 2 hits
# trkhits = trkhits.groupby(level=[0,1]).apply(lambda x: x.iloc[2:]).reset_index(level=2, drop=True)
# trkhits = trkhits[trkhits.rr > 0.3]
trkhits = trkhits

In [ ]:
# save root file
output = uproot.recreate(os.path.join(savepath, calibdfname))

output["caloskim/hits"] = {
    "trk.hits2.h.dedx":           np.array(trkhits.dedx),
    "trk.hits2.dqdx":             np.array(trkhits.dqdx),   
    "trk.hits2.h.pitch":          np.array(trkhits.pitch),   
    "trk.hits2.h.integral":       np.array(trkhits.integral),      
    "trk.hits2.rr":               np.array(trkhits.rr), 
    "trk.hits2.pitch":            np.array(trkhits.pitch),
    "trk.hits2.h.wire":           np.array(trkhits.wire),  
    "trk.hits2.h.tpc":            np.array(trkhits.tpc),   
    "trk.hits2.h.sumadc":         np.array(trkhits.sumadc),   
    "trk.hits2.h.time":           np.array(trkhits.t), 
    "trk.hits2.h.sp.x":           np.array(trkhits.x),    
    "trk.hits2.h.sp.y":           np.array(trkhits.y),     
    "trk.hits2.h.sp.z":           np.array(trkhits.z),       
    "trk.hits2.h.truth.h_e":      np.array(trkhits.truth.h_e),         
    "trk.hits2.h.truth.h_nelec":  np.array(trkhits.truth.h_nelec),        
    "trk.hits2.h.truth.pitch":    np.array(trkhits.truth.pitch),       
    "trk.hits2.h.truth.rr":       np.array(trkhits.truth.rr),     
    }

output["caloskim/trk"] = {
    "trk.start.x":                np.array(trks.start.x),  
    "trk.start.y":                np.array(trks.start.y), 
    "trk.start.z":                np.array(trks.start.z),      
    "trk.end.x":                  np.array(trks.end.x),  
    "trk.end.y":                  np.array(trks.end.y), 
    "trk.end.z":                  np.array(trks.end.z),      
    "trk.dir.x":                  np.array(trks.dir.x),  
    "trk.dir.y":                  np.array(trks.dir.y), 
    "trk.dir.z":                  np.array(trks.dir.z),      
    "trk.len":                    np.array(trks.len),      
    # "trk.truth.p.start.x":        np.array(trks.truth.p.start.x),    
    # "trk.truth.p.start.y":        np.array(trks.truth.p.start.y),   
    # "trk.truth.p.start.z":        np.array(trks.truth.p.start.z),    
    # "trk.truth.p.end.x":          np.array(trks.truth.p.end.x),  
    # "trk.truth.p.end.y":          np.array(trks.truth.p.end.y),  
    # "trk.truth.p.end.z":          np.array(trks.truth.p.end.z),  
    # "trk.truth.p.end_process":    np.array(trks.truth.p.end_process),       
    # "trk.truth.p.pdg":            np.array(trks.truth.p.end_process),       
    }